# Instalações


In [1]:
%%capture
!pip install TTS

In [2]:
%%capture
!sudo apt-get install espeak

# Bibliotecas

In [3]:
# Gerais
import os
import glob
from google.colab import drive

# TTS - Dataset
from TTS.utils.audio import AudioProcessor
from TTS.tts.datasets import load_tts_samples
from TTS.tts.configs.shared_configs import BaseDatasetConfig

# TTS - Modelo
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer

# Conexão ao Drive

In [4]:
# Conectar ao Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Modificar path de trabalho do notebook
try:
  os.chdir("./drive/")
except:
  pass

# Funções - Dataset

In [6]:
# custom formatter implementation
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "harbinger"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0]) + '.wav'
            text = cols[1]
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [7]:
def tratar_audios_vits(lista_audios, output_path):
  # Definir AP
  ap = AudioProcessor(sample_rate=22050,
                      resample=False,
                      num_mels=80,
                      log_func='np.log10',
                      fft_size=1024,
                      base=10,
                      hop_length=256,
                      win_length=1024)
  l = len(lista_audios)
  for idx, INPUT_WAV_FILE in enumerate(lista_audios):
    # Montar caminho de saida
    OUTPUT_WAV_FILE = output_path + '/' + INPUT_WAV_FILE.split('/')[-1]
    # Carregar
    wav = ap.load_wav(INPUT_WAV_FILE, sr=ap.sample_rate)
    # Salvar
    ap.save_wav(wav, path=OUTPUT_WAV_FILE, sr=ap.sample_rate)
    # Report
    if idx != 0 and idx % 100 == 0:
      print(f'> {idx} of {l}')

# Execução

**Fonte para tratamento de áudio [aqui](#https://docs.coqui.ai/en/latest/main_classes/audio_processor.html#TTS.utils.audio.AudioProcessor.load_stats)**

In [8]:
# Tratar aúdios - Quando necessário. Executar apenas uma vez
#input_path_audios = 'PATH_ENTRADA_AUDIOS_AQUI'
#lista_path_audios = list(sorted(glob.glob(input_path_audios)))
#output_path = 'PATH_SAIDA_AUDIOS_AQUI'

# Executar tratamento
#tratar_audios_vits(lista_path_audios, output_path)

In [9]:
# dataset config for one of the pre-defined datasets
#dataset_config = BaseDatasetConfig(meta_file_train='metadata.txt', language="en-us", path='PATH_DRIVE')
# load training samples
#train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, eval_split_size=0.15, formatter=formatter)

# Execução - Fine-Tuning VITS

### Variáveis e Caminhos

In [10]:
# Definir caminhos de ambiente
output_path = 'PATH_DRIVE_PARA_SALVAR_TREINO'
phoneme_path_output = os.path.join(output_path, "phoneme_cache")

# Definir variáveis
run_name = 'vits_harbinger_ft_v2'
EPOCHS = 1000
BATCH_SIZE = 32
BATCH_SIZE_EVAL = 16
LEARNING_RATE = 0.001
USE_PHONEME = False

### Configurações

In [11]:
# Definir configuração - dataset formato
dataset_config = BaseDatasetConfig(meta_file_train='metadata.txt',
                                   language="en-us",
                                   path='PATH_DRIVE')

# Definir configuração - aúdio
audio_config = VitsAudioConfig(sample_rate=22050,
                               win_length=1024,
                               hop_length=256,
                               num_mels=80,
                               mel_fmin=0,
                               mel_fmax=None)

In [12]:
# Definir configuração - modelo
model_config = VitsConfig(audio=audio_config,
                          lr=LEARNING_RATE,
                          run_name=run_name,
                          save_on_interrupt=True,
                          save_n_checkpoints=10,
                          save_checkpoints=True,
                          save_all_best=False,
                          save_step=1500,
                          save_best_after=5000,
                          batch_size=BATCH_SIZE,
                          training_seed=1206,
                          eval_batch_size=BATCH_SIZE_EVAL,
                          batch_group_size=5,
                          num_loader_workers=2,
                          num_eval_loader_workers=1,
                          run_eval=True,
                          test_delay_epochs=-1,
                          epochs=EPOCHS,
                          text_cleaner="english_cleaners",
                          use_phonemes=USE_PHONEME,
                          phoneme_language="en-us",
                          phoneme_cache_path=phoneme_path_output,
                          #target_loss="loss_1",
                          compute_input_seq_cache=True,
                          print_step=500,
                          print_eval=True,
                          mixed_precision=True,
                          output_path=output_path,
                          datasets=[dataset_config],
                          cudnn_benchmark=False)

In [13]:
# Inicializar - Amostras
train_samples, eval_samples = load_tts_samples(dataset_config,
                                               eval_split=True,
                                               eval_split_size=0.10,
                                               formatter=formatter)

# Inicializar - Audio Processor
ap = AudioProcessor.init_from_config(model_config)

# Inicializar - Tokenizer
tokenizer, model_config = TTSTokenizer.init_from_config(model_config)

# Inicializar - Modelo
model_tts = Vits(model_config, ap, tokenizer, speaker_manager=None)

 | > Found 990 files in /content/drive/MyDrive/HSDataset
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [14]:
# Inicializar - Trainer
checkpoint_path = 'PATH_CHECKPOINT_PRIMEIRO_RESTORE_TEM_QUE_SER_DO_MODELO_BASE'
trainer = Trainer(TrainerArgs(restore_path=checkpoint_path),
                  model_config,
                  output_path,
                  model=model_tts,
                  train_samples=train_samples,
                  eval_samples=eval_samples)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 1206
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/VITS/vits_harbinger_ft_v2-January-25-2024_07+14PM-0000000
 > Restoring from checkpoint_1047238.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 1047238

 > Model has 83046892 parameters


In [ ]:
# Realizer Fine-Tuning
trainer.fit();


 > EPOCH: 0/1000
 --> /content/drive/MyDrive/VITS/vits_harbinger_ft_v2-January-25-2024_07+14PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 891



 > TRAINING (2024-01-25 19:15:04) 


 | > Preprocessing samples
 | > Max text length: 211
 | > Min text length: 7
 | > Avg text length: 64.79124579124579
 | 
 | > Max audio length: 366982.0
 | > Min audio length: 29783.0
 | > Avg audio length: 130224.82154882155
 | > Num. instances discarded samples: 0
 | > Batch group size: 160.


/usr/local/lib/python3.10/dist-packages/torch/functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:863.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]

 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.01785918076833089 (+0.0005060434341430664)
     | > avg_loss_disc: 1.8575935165087383 (-0.1905911167462666)
     | > avg_loss_disc_real_0: 0.10015721246600151 (+0.022920117403070137)
     | > avg_loss_disc_real_1: 0.14877436930934587 (-0.028169921288887678)
     | > avg_loss_disc_real_2: 0.1674948384364446 (-0.06747645884752274)
     | > avg_loss_disc_real_3: 0.21471243351697922 (+0.043016597628593445)
     | > avg_loss_disc_real_4: 0.14720221484700838 (-0.03574320301413536)
     | > avg_loss_disc_real_5: 0.19579264024893442 (-0.03303308536609015)
     | > avg_loss_0: 1.8575935165087383 (-0.1905911167462666)
     | > avg_loss_gen: 3.09318470954895 (+0.014196793238321792)
     | > avg_loss_kl: 1.1284720202287037 (+0.09294175108273817)
     | > avg_loss_feat: 16.613197167714436 (+0.4206388791402169)
     | > avg_loss_mel: 16.109729766845703 (+0.8646505673726406)
     | > avg_loss_duration: 1.2585156758626301 (+0.001986205577850342)
    

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.01860467592875163 (+0.000745495160420738)
     | > avg_loss_disc: 1.9417322278022766 (+0.08413871129353834)
     | > avg_loss_disc_real_0: 0.08652660250663757 (-0.013630609959363937)
     | > avg_loss_disc_real_1: 0.1574979449311892 (+0.008723575621843338)
     | > avg_loss_disc_real_2: 0.19625151654084524 (+0.028756678104400635)
     | > avg_loss_disc_real_3: 0.21627072244882584 (+0.0015582889318466187)
     | > avg_loss_disc_real_4: 0.16047408680121103 (+0.013271871954202652)
     | > avg_loss_disc_real_5: 0.22866162657737732 (+0.0328689863284429)
     | > avg_loss_0: 1.9417322278022766 (+0.08413871129353834)
     | > avg_loss_gen: 3.2046597401301065 (+0.11147503058115626)
     | > avg_loss_kl: 1.050417830546697 (-0.07805418968200684)
     | > avg_loss_feat: 17.456520239512127 (+0.8433230717976912)
     | > avg_loss_mel: 15.813745498657227 (-0.29598426818847656)
     | > avg_loss_duration: 1.254426379998525 (-0.004089295864105225)
 

 | > Synthesizing test sentences.


Exception ignored in: <function _xla_gc_callback at 0x7cfc63803ac0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 

  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.019217530886332195 (+0.0006128549575805664)
     | > avg_loss_disc: 1.9584777156511943 (+0.016745487848917717)
     | > avg_loss_disc_real_0: 0.0896065104752779 (+0.0030799079686403275)
     | > avg_loss_disc_real_1: 0.14188436542948088 (-0.01561357950170833)
     | > avg_loss_disc_real_2: 0.21037301421165466 (+0.014121497670809419)
     | > avg_loss_disc_real_3: 0.1972386265794436 (-0.019032095869382232)
     | > avg_loss_disc_real_4: 0.18449883411327997 (+0.02402474731206894)
     | > avg_loss_disc_real_5: 0.1940168042977651 (-0.034644822279612214)
     | > avg_loss_0: 1.9584777156511943 (+0.016745487848917717)
     | > avg_loss_gen: 3.034550944964091 (-0.17010879516601562)
   